# Satellite Image Processing for Deforestation Detection

## Overview
This notebook implements a comprehensive deforestation detection system using Google Earth Engine (GEE) and Sentinel-2 imagery. It focuses on the Córdoba region in Argentina, employing multiple filtering techniques and NDVI-based change detection.

## Technical Implementation Details

### 1. Data Collection & Filtering
- **Data Source**: 
  - Sentinel-2 Collection: 'COPERNICUS/S2'
  - Spatial Resolution: 10 meters
  - Spectral Bands: B2, B3, B4, B8

- **Filtering Chain**:
  ```python
  ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(region)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .median()
  ```
  1. Spatial Filter: `.filterBounds(region)`
  2. Temporal Filter: `.filterDate(start_date, end_date)`
  3. Cloud Filter: `CLOUDY_PIXEL_PERCENTAGE < 20%`
  4. Composite Filter: `.median()` for temporal aggregation

### 2. NDVI Calculation & Change Detection
- **NDVI Formula**: 
  ```python
  ndvi = image.normalizedDifference(['B8', 'B4'])
  # Where: B8 = NIR band, B4 = Red band
  ```

- **Change Detection Parameters**:
  - NDVI Threshold: -0.1 (configurable)
  - Change Detection: `ndvi_after.subtract(ndvi_before)`
  - Deforestation Mask: `ndvi_change.lt(ndvi_threshold)`

### 3. Area Calculations
- **Pixel Area Conversion**:
  ```python
  deforestation_area = deforestation.multiply(ee.Image.pixelArea())
  ```
- **Scale**: 10 meters
- **Units**: Hectares (area/10000)
- **Maximum Pixels**: 1e9 for computation

### 4. Visualization Parameters
- **True Color Display**:
  ```python
  true_color_viz = {
      'bands': ['B4', 'B3', 'B2'],
      'min': 0,
      'max': 3000
  }
  ```

- **NDVI Display**:
  ```python
  ndvi_viz = {
      'min': -1,
      'max': 1,
      'palette': ['blue', 'white', 'green']
  }
  ```

- **Change Detection**:
  ```python
  change_viz = {
      'min': -0.3,
      'max': 0.3,
      'palette': ['red', 'yellow', 'green']
  }
  ```

## Processing Steps
1. **Pre-processing**:
   - Spatial boundary definition
   - Date range selection
   - Cloud masking application

2. **NDVI Processing**:
   - Band selection (NIR, Red)
   - NDVI calculation
   - Temporal differencing

3. **Change Detection**:
   - Threshold application
   - Area calculation
   - Mask generation

4. **Results Generation**:
   - Area statistics
   - Duration calculation
   - Report compilation
   - Map visualization

## Output Analysis
1. **Quantitative Metrics**:
   ```
   - Total Area (hectares)
   - Deforested Area (hectares)
   - Analysis Duration (days)
   - Cloud Coverage (percentage)
   ```

2. **Visual Outputs**:
   - Before/After satellite imagery
   - NDVI maps
   - Change detection visualization
   - Deforestation mask

In [13]:
import ee
import geemap
import folium
from folium import plugins


In [14]:
ee.Authenticate(auth_mode='colab')

True

In [15]:
ee.Initialize(project='ee-mujtabanaqvi29')

In [16]:
import ee
import geemap
from datetime import datetime  
   


In [17]:
def detect_deforestation_with_filters(region, start_date_before, end_date_after, ndvi_threshold=-0.15):
    """
    Detects deforestation, applies Gaussian smoothing and Sobel edge detection, 
    and generates a detailed report including:
    - Dates of analysis
    - Duration
    - Total investigated area
    - Total deforested area (in hectares)
    """
    # Load Sentinel-2 imagery
    before = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(region) \
        .filterDate(start_date_before, end_date_after) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .median()

    after = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(region) \
        .filterDate(end_date_after, '2023-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .median()

    # Calculate NDVI for before and after
    ndvi_before = before.normalizedDifference(['B8', 'B4']).rename('NDVI_Before')
    ndvi_after = after.normalizedDifference(['B8', 'B4']).rename('NDVI_After')

    # Apply Gaussian smoothing
    gaussian_kernel = ee.Kernel.gaussian(radius=3, sigma=2, units='pixels')
    smoothed_ndvi_before = ndvi_before.convolve(gaussian_kernel).rename('Smoothed_NDVI_Before')
    smoothed_ndvi_after = ndvi_after.convolve(gaussian_kernel).rename('Smoothed_NDVI_After')

    # Sobel edge detection
    sobel_kernel = ee.Kernel.sobel()
    edges_before = ndvi_before.convolve(sobel_kernel).rename('Edges_Before')
    edges_after = ndvi_after.convolve(sobel_kernel).rename('Edges_After')

    # Calculate NDVI change
    ndvi_change = ndvi_after.subtract(ndvi_before).rename('NDVI_Change')

    # Apply threshold to detect deforestation
    deforestation = ndvi_change.lt(ndvi_threshold).selfMask()

    # Calculate the total deforested area
    deforestation_area = deforestation.multiply(ee.Image.pixelArea())
    total_deforestation_area = deforestation_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=10,
        maxPixels=1e9
    ).getInfo()

    # Convert to hectares
    deforested_area_ha = total_deforestation_area.get('NDVI_Change', 0) / 10_000

    # Calculate the total area of the region (in hectares)
    total_region_area = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    total_region_area_ha = total_region_area.get('area') / 10_000

    # Calculate the duration of analysis
    start_date = datetime.strptime(start_date_before, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_after, '%Y-%m-%d')
    duration_days = (end_date - start_date).days

    # Visualization parameters
    ndvi_viz = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
    change_viz = {'min': -0.3, 'max': 0.3, 'palette': ['red', 'yellow', 'green']}
    deforestation_viz = {'min': 0, 'max': 1, 'palette': ['red']}
    true_color_viz = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
    edge_viz = {'min': 0, 'max': 1, 'palette': ['black', 'white']}

    # Visualization
    Map = geemap.Map()
    Map.centerObject(region, 12)
    Map.addLayer(before, true_color_viz, 'Satellite Image Before')
    Map.addLayer(after, true_color_viz, 'Satellite Image After')
    Map.addLayer(ndvi_before, ndvi_viz, 'NDVI Before')
    Map.addLayer(ndvi_after, ndvi_viz, 'NDVI After')
    Map.addLayer(smoothed_ndvi_before, ndvi_viz, 'Smoothed NDVI Before')
    Map.addLayer(smoothed_ndvi_after, ndvi_viz, 'Smoothed NDVI After')
    Map.addLayer(edges_before, edge_viz, 'Edges Before')
    Map.addLayer(edges_after, edge_viz, 'Edges After')
    Map.addLayer(ndvi_change, change_viz, 'NDVI Change')
    Map.addLayer(deforestation, deforestation_viz, 'Deforestation Detected')

    # Generate the detailed report
    report = f"""
    Deforestation Analysis Report
    ------------------------------
    Region Investigated: Custom Region (User Defined)
    Total Area Investigated: {total_region_area_ha:.2f} hectares
    Analysis Dates:
        - Before: {start_date_before}
        - After: {end_date_after}
    Analysis Duration: {duration_days} days
    NDVI Threshold for Deforestation: {ndvi_threshold}
    Total Deforested Area: {deforested_area_ha:.2f} hectares
    """
    print(report)
    return Map

# Define the region (example: Córdoba region)
region = ee.Geometry.Polygon([
    [[-64.18, -31.40], [-64.17, -31.40], [-64.17, -31.41], [-64.18, -31.41], [-64.18, -31.40]]
])

# Detect deforestation with additional filters and generate the report
Map = detect_deforestation_with_filters(region, '2022-01-01', '2022-12-31', ndvi_threshold=-0.1)
Map



    Deforestation Analysis Report
    ------------------------------
    Region Investigated: Custom Region (User Defined)
    Total Area Investigated: 105.37 hectares
    Analysis Dates:
        - Before: 2022-01-01
        - After: 2022-12-31
    Analysis Duration: 364 days
    NDVI Threshold for Deforestation: -0.1
    Total Deforested Area: 0.12 hectares
    


Map(center=[-31.405000008271625, -64.1750000000164], controls=(WidgetControl(options=['position', 'transparent…